In [1]:
from flask import Flask, Response, render_template, request, jsonify, send_from_directory, redirect, url_for, flash, url_for, session
import cv2
from werkzeug.utils import secure_filename
import numpy as np
import base64
from keras.callbacks import LambdaCallback
from io import BytesIO
from functools import wraps
from PIL import Image
from keras.models import load_model
import joblib
from datetime import datetime
import mysql.connector
from mysql.connector import Error
import matplotlib.pyplot as plt
import datetime
import os
from keras.preprocessing.image import ImageDataGenerator
import mediapipe as mp
import pickle
from sklearn.model_selection import train_test_split
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D, Flatten, Dense
from keras.optimizers import Adam
from keras.utils import to_categorical
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder

2023-05-30 21:17:29.214347: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2023-05-30 21:17:29.666563: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2023-05-30 21:17:29.668017: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-05-30 21:17:31.829832: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


In [2]:
import mediapipe as mp
import cv2

mp_face_detection = mp.solutions.face_detection

def crop_face(image_path):
    # Load gambar
    image = cv2.imread(image_path)
    # Deteksi wajah pada gambar
    with mp_face_detection.FaceDetection(min_detection_confidence=0.5) as face_detection:
        results = face_detection.process(image)
        if results.detections:
            for detection in results.detections:
                bbox = detection.location_data.relative_bounding_box
                h, w, c = image.shape
                x = int(bbox.xmin * w)
                y = int(bbox.ymin * h)
                width = int(bbox.width * w)
                height = int(bbox.height * h)
                crop_img = image[y:y+height, x:x+width]
    return crop_img


In [22]:
def predict_image(image, model):
    img = cv2.resize(image, (150, 150), interpolation = cv2.INTER_AREA)
    img = np.array([img], dtype=np.int16)  # Add an extra dimension to the image
    prediction = model.predict(img)
    predicted_label = np.argmax(prediction)
    predicted_score = prediction[0][predicted_label]
    return predicted_label, predicted_score


model = load_model('model.h5')

In [7]:
data_dir = 'Dataset/'
import os
image_files = [os.path.join(data_dir, f) for f in os.listdir(data_dir) if f.endswith('.jpeg')]

In [23]:
print(image_files[0])
crop_image = crop_face(image_files[0])
lab, score = predict_image(crop_image,model)
print(f"label : {lab}\nscore : {score}")

Dataset/4.jpeg
1/1 [==============================] - 0s 238ms/step
label : 4
score : 1.0


In [13]:
import pandas as pd

data = image_files
predict = []
for i in range(len(data)):
    crop_image = crop_face(image_files[0])
    lab, score = predict_image(crop_image,model)
    predict.append(lab)

1/1 [==============================] - 0s 47ms/step


In [21]:
df = pd.DataFrame()
df['data'] = data
df['label'] = [str(data[i]).replace('Dataset/','') for i in range(len(df))]
df['label'] = [str(df['label'][i]).replace('.jpeg','') for i in range(len(df))]
df['predict_result'] = predict
df

,data,label,predict_result
0,Dataset/4.jpeg,4,4
1,Dataset/5.jpeg,5,4
2,Dataset/2.jpeg,2,4
3,Dataset/6.jpeg,6,4
4,Dataset/7.jpeg,7,4
5,Dataset/11.jpeg,11,4
6,Dataset/3.jpeg,3,4
7,Dataset/9.jpeg,9,4
8,Dataset/8.jpeg,8,4
9,Dataset/10.jpeg,10,4
